In [ ]:
!pip install pyteomics

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyteomics import mzxml
from google.colab import files

#uploaded file path
def upload_files():
    uploaded = files.upload()
    return list(uploaded.keys())[0]


#mzxml 파일 load
def load_data(filepath):

    mzXML_data = mzxml.MzXML(filepath)
    columns = ['msLevel', 'peak', 'retentionTime', 'precursor',
               'totIonCurrent']
    data = pd.DataFrame(np.zeros([len(mzXML_data), len(columns)]),
                        columns=columns)

    ms_level_list = []
    peak_list = []
    retention_time_list = []
    precursor_list = []
    tot_ion_current_list = []

    for idx, tmp_data in enumerate(mzXML_data):
        ms_level = tmp_data['msLevel']
        mz_array, intensity_array = tmp_data['m/z array'], tmp_data['intensity array']
        peak = np.concatenate((np.expand_dims(mz_array, axis=1),
                               np.expand_dims(intensity_array, axis=1)), axis=1)
        retention_time = tmp_data['retentionTime']
        precursor = None
        tot_ion_current = tmp_data.get('totIonCurrent')

        if ms_level == 2:
            precursor = tmp_data['precursorMz'][0]['precursorMz']

        ms_level_list.append(ms_level)
        peak_list.append(peak)
        retention_time_list.append(retention_time)
        precursor_list.append(precursor)
        tot_ion_current_list.append(tot_ion_current)

    data['msLevel'] = ms_level_list
    data['peak'] = peak_list
    data['retentionTime'] = retention_time_list
    data['precursor'] = precursor_list
    data['totIonCurrent'] = tot_ion_current_list

    return data


#mzxml 이용하여 LC chromatogram 시각화
def plot_chromatogram (data):

    fig, ax = plt.subplots(figsize=(15, 5))
    x = data['retentionTime']
    y= data['totIonCurrent']
    max_totIonCurrent = max(data['totIonCurrent'])
    y = [100 * value / max_totIonCurrent if pd.notna(value) else np.nan for value in data['totIonCurrent']]
    ax.plot(x, y, linestyle='-',marker='', color='#173F5F')
    ax.set_title('Total Ion Chromatogram')
    ax.set_xlabel('Time (min)')
    ax.set_ylabel('Relative Intensity')
    plt.show()
    return fig, ax




# 특정 RT를 입력해주면, 해당 RT에서의 매스스펙트럼 시각화
def plot_MS(data,rt):

    closest_rt = min(data['retentionTime'], key=lambda x: abs(x - rt))
    clicked_data = data[data['retentionTime'] == closest_rt].iloc[0]
    mz_array = clicked_data['peak'][:,0]
    intensity_array = clicked_data['peak'][:,1]
    normalized_level = np.max(intensity_array)
    if normalized_level != 0:
        intensity_array = 100 * intensity_array / normalized_level

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.vlines(mz_array, 0, intensity_array, color='#173F5F')
    ax.set_title(f'Spectrum at RT {clicked_data["retentionTime"]} min')
    ax.set_xlabel('m/z')
    ax.set_ylabel('Relative Intensity')
    plt.show()
    return fig, ax



filepath = upload_files()
data =load_data(filepath)
fig, ax =plot_chromatogram(data)
rt = float(input("분석하고 싶은 RT를 입력하세요: "))
fig, ax = plot_MS(data, rt)